### Create gold view

These view will be utilized to create reports

In [0]:
USE CATALOG imdb_dev;

In [0]:
SELECT current_catalog();

##### Create view `vw_titles_count`


In [0]:
CREATE OR REPLACE VIEW imdb_dev.gold.vw_titles_count AS
SELECT COUNT(*) AS total_titles
  FROM imdb_dev.gold.dim_title;

##### Create view `vw_actors_count`

In [0]:
CREATE OR REPLACE VIEW imdb_dev.gold.vw_actors_count AS
SELECT COUNT(DISTINCT nconst) AS actors_count
  FROM imdb_dev.gold.bridge_title_person
 WHERE role = 'actor';

##### Create view `vw_actress_count`

In [0]:
CREATE OR REPLACE VIEW imdb_dev.gold.vw_actress_count AS
SELECT COUNT(DISTINCT nconst) AS actress_count
  FROM imdb_dev.gold.bridge_title_person
 WHERE role = 'actress';

##### Create view `vw_directors_count`

In [0]:

CREATE OR REPLACE VIEW imdb_dev.gold.vw_directors_count AS
SELECT COUNT(DISTINCT nconst) AS directors_count
  FROM imdb_dev.gold.bridge_title_person
 WHERE role = 'director';

##### Create view `vw_rating_genres`

In [0]:
CREATE OR REPLACE VIEW imdb_dev.gold.vw_rating_genres AS
SELECT 
    genre,
    ROUND(AVG(average_rating), 1) AS avg_rating,
    COUNT(*) AS num_titles,
    SUM(num_votes) AS total_votes,
    ROUND(SUM(num_votes)/COUNT(*),1) AS avg_votes
FROM imdb_dev.gold.dim_title 
LATERAL VIEW explode(genres) AS genre
WHERE num_votes >= 1000 -- Use only movies with more than 1000 votes
GROUP BY genre
ORDER BY avg_rating DESC;


##### Create view `vw_rating_year`

In [0]:
CREATE OR REPLACE VIEW imdb_dev.gold.vw_rating_year AS
SELECT 
    year,
    ROUND(AVG(average_rating), 1) AS avg_rating,
    COUNT(*) AS num_titles,
    SUM(num_votes) AS total_votes,
    ROUND(SUM(num_votes)/COUNT(*),1) AS avg_votes
FROM imdb_dev.gold.dim_title 
WHERE num_votes >= 1000  -- Use only movies with more than 1000 votes
GROUP BY year
ORDER BY year ASC;

##### Create view `vw_rating_year_genre`

In [0]:
CREATE OR REPLACE VIEW imdb_dev.gold.vw_rating_year_genre AS
SELECT 
    year,
    genre,
    ROUND(AVG(average_rating), 1) AS avg_rating,
    COUNT(*) AS num_titles,
    SUM(num_votes) AS total_votes,
    ROUND(SUM(num_votes)/COUNT(*),1) AS avg_votes
FROM imdb_dev.gold.dim_title 
LATERAL VIEW explode(genres) AS genre
WHERE num_votes >= 1000 -- Use only movies with more than 1000 votes
GROUP BY genre, year
ORDER BY year DESC;

##### Create view 
1. `vw_top_rated_directors` 
2. `vw_top_rated_actor` 
3. `vw_top_rated_actress`

Filter person that participated in more than 10 movies with more than 1000 votes.

In [0]:
CREATE OR REPLACE VIEW imdb_dev.gold.vw_top_rated_directors AS
SELECT p.name,
       ROUND(AVG(t.average_rating),1) AS avg_rating,
       COUNT(*) AS num_titles
  FROM imdb_dev.gold.dim_person p
  JOIN imdb_dev.gold.bridge_title_person tp ON p.nconst = tp.nconst
  JOIN imdb_dev.gold.dim_title t ON tp.tconst = t.tconst
 WHERE tp.role = 'director' AND t.num_votes >= 1000 -- Use only movies with more than 1000 votes
GROUP BY p.nconst, p.name
HAVING num_titles > 10 -- Use only people with more than 10 movies
ORDER BY avg_rating DESC;

       

In [0]:
CREATE OR REPLACE VIEW imdb_dev.gold.vw_top_rated_actors AS
SELECT p.name,
       ROUND(AVG(t.average_rating),1) AS avg_rating,
       COUNT(*) AS num_titles
  FROM imdb_dev.gold.dim_person p
  JOIN imdb_dev.gold.bridge_title_person tp ON p.nconst = tp.nconst
  JOIN imdb_dev.gold.dim_title t ON tp.tconst = t.tconst
 WHERE tp.role = 'actor' AND t.num_votes >= 1000 -- Use only movies with more than 1000 votes
GROUP BY p.nconst, p.name
HAVING num_titles > 10
ORDER BY avg_rating DESC;

In [0]:
CREATE OR REPLACE VIEW imdb_dev.gold.vw_top_rated_actress AS
SELECT p.name,
       ROUND(AVG(t.average_rating),1) AS avg_rating,
       COUNT(*) AS num_titles
  FROM imdb_dev.gold.dim_person p
  JOIN imdb_dev.gold.bridge_title_person tp ON p.nconst = tp.nconst
  JOIN imdb_dev.gold.dim_title t ON tp.tconst = t.tconst
 WHERE tp.role = 'actress' AND t.num_votes >= 1000 -- Use only movies with more than 1000 votes
GROUP BY p.nconst, p.name
HAVING num_titles > 10
ORDER BY avg_rating DESC;